In [4]:
import sys, os
cwd_split = os.getcwd().split(os.sep)
root = os.sep.join(cwd_split[:cwd_split.index('mitsuba3-sensor')])
sys.path.insert(0, f"{root}/mitsuba3-sensor/build/python")

In [5]:
import drjit as dr
import mitsuba as mi
import os

from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy as np

mi.set_variant('cuda_ad_rgb')

import integrator.camera

# Scene

In [6]:
from mitsuba.scalar_rgb import Transform4f as T
integrator = {
    # 'type': 'direct_projective',
    'type': 'camera'
    # 'type': 'camera',
    # 'type' : 'prb_projective'
    # 'type': 'direct_reparam'
}

# integrator = mi.load_dict({
#     # 'type': 'direct_projective',
#     'type': 'camera_test',
#     'sppc': 1024,
#     'sppp': 1024,
#     'sppi': 0,
# })

img_size = 512

sensor_to_world = T.look_at(
                        origin=(0, 0, 2),
                        target=(0, 0, 0),
                        up=(0, 1, 0)
                    )

scene = mi.load_dict({
    'type': 'scene',
    'integrator': integrator,
    'sensor':  {
        'type': 'perspective',
        'to_world': sensor_to_world,
        'fov': 60,
        'film': {
            'type': 'hdrfilm',
            'width': img_size,
            'height': img_size,
            'rfilter': { 'type': 'gaussian' },
            'sample_border': True
        },
    },
    'wall': {
        'type': 'rectangle',
        'to_world': T.translate([0, 0, 0]).scale(2),
        'bsdf': {
            'type': 'diffuse',
            'reflectance': { 'type': 'rgb', 'value': (0.5, 0.5, 0.5) },
        }
    },
    # 'block': {
    #     'type': 'sphere',
    #     'to_world': mi.ScalarTransform4f.scale([0.5, 0.5, 0.5]).translate([0, 0, -1]),
    #     'bsdf': {
    #         'type': 'diffuse'
    #     }
    # },
    'block': {
        'type': 'sphere',
        'to_world': T.translate([0, 0, 1]).scale(0.3),
        'bsdf': {
            'type': 'diffuse'
        }
        # 'emitter': {
        #     'type': 'area',
        #     'radiance': {'type': 'rgb', 'value': [10, 10, 10]}
        # },
    },
    'light': {
        'type': 'disk',
        'emitter': {
            'type': 'area',
            'radiance': {'type': 'rgb', 'value': [10, 10, 10]}
        },
        'to_world': T.look_at(
                        origin=(1, 1, 3),
                        target=(0, 0, 0),
                        up=(0, 1, 0)
                    ).scale(0.5),
    }
})

In [7]:
params = mi.traverse(scene)
key = 'sensor.to_world'

def apply_trafo(params, value):
    trafo = mi.Transform4f.translate([value, 0.0, 0.0])
    # trafo = mi.Transform4f.translate([0.0, 0.0, 0.0]).rotate([0, 0, 1], value)
    # trafo = mi.Transform4f.translate([0.0, 0.0, value])

    params[key] = trafo @ sensor_to_world
    params.update()

In [8]:
img_ref = mi.render(scene, seed=0, spp=1024)
mi.Bitmap(img_ref).write("output/primal.exr")

# Gradient

In [9]:
# Finite differences
epsilon = 5e-3
fd_spp = 512
fd_repeat = 4

img1 = dr.zeros(mi.TensorXf, (img_size, img_size, 3))
img2 = dr.zeros(mi.TensorXf, (img_size, img_size, 3))

for it in range(fd_repeat):
    apply_trafo(params, -epsilon)
    img1 += mi.render(scene, spp=fd_spp, seed=it)
    params.update()

    apply_trafo(params, +epsilon)
    img2 += mi.render(scene, spp=fd_spp, seed=it)
    params.update()

    print(f"{it+1}/{fd_repeat}", end='\r')

img_fd = (img2 - img1) / (epsilon*2) / fd_repeat
mi.Bitmap(img_fd).write("output/grad_fd.exr")

# Prb

In [10]:
x = mi.Float(0.0)
dr.enable_grad(x)
apply_trafo(params, x)
dr.forward(x, dr.ADFlag.ClearEdges)

image = mi.render(scene, params = params, spp=1024)

Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [512, 512],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 768 KiB of image data ]
]

In [12]:
img_guided = dr.forward_to(image)
mi.Bitmap(img_guided).write("output/grad.exr")